In [2]:

%load_ext autoreload
%autoreload 2

import re

import numpy as np

import torch

from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from torch.utils.data import DataLoader, Dataset, random_split
from torch.utils.tensorboard import SummaryWriter
from tqdm import trange
import seaborn as sns
import networks
from signal_dataset import SignalDataset
import tsaug
import torchaudio.transforms as T

import torchaudio
import ray
from ray import train, tune
from ray.train import Checkpoint
from ray.tune.schedulers import ASHAScheduler
from IPython.display import Audio
from matplotlib.patches import Rectangle
from signal_model import SignalModel, NeuroNet, load_yaml
from pathlib import Path
from sklearn.model_selection import KFold
import yaml

In [3]:
loaded_signal = []
sample_rate = 1562500
channel_1 = 'ch1'
channel_2 = 'ch2'
channel_3 = 'ch3'
signal_data_dir = "/mnt/home2/Motor_project/AE_PETR_loziska/"
train_config = [{"label": (int(i.stem)-1)//4,
              "channels": len(list(i.glob('*' + channel_1 + '.bin'))),
              "interval": [0, int(4.5*sample_rate)],
              "bin_path": list(i.glob('*' + channel_1 + '.bin'))[0]}
             for i in Path(signal_data_dir).glob('*') if re.search(r'\d$', i.stem)] + [{
              "label": (int(i.stem)-1)//4,
              "channels": len(list(i.glob('*' + channel_2 + '.bin'))),
              "interval": [0, int(4.5*sample_rate)],
              "bin_path": list(i.glob('*' + channel_2 + '.bin'))[0]}
             for i in Path(signal_data_dir).glob('*') if re.search(r'\d$', i.stem)]

test_config = [{"label": (int(i.stem)-1)//4,
              "channels": len(list(i.glob('*' + channel_3 + '.bin'))),
              "interval": [int(4.5*sample_rate), 5*sample_rate],
              "bin_path": list(i.glob('*' + channel_3 + '.bin'))[0]}
             for i in Path(signal_data_dir).glob('*') if re.search(r'\d$', i.stem)]

In [7]:
train_config

[{'label': 3,
  'channels': 1,
  'interval': [0, 7031250],
  'bin_path': PosixPath('/mnt/home2/Motor_project/AE_PETR_loziska/15/AE-DATA-conti-7394086577130671-ch1.bin')},
 {'label': 2,
  'channels': 1,
  'interval': [0, 7031250],
  'bin_path': PosixPath('/mnt/home2/Motor_project/AE_PETR_loziska/10/AE-DATA-conti-7394027167083912-ch1.bin')},
 {'label': 4,
  'channels': 1,
  'interval': [0, 7031250],
  'bin_path': PosixPath('/mnt/home2/Motor_project/AE_PETR_loziska/18/AE-DATA-conti-7394027310105093-ch1.bin')},
 {'label': 0,
  'channels': 1,
  'interval': [0, 7031250],
  'bin_path': PosixPath('/mnt/home2/Motor_project/AE_PETR_loziska/03/AE-DATA-conti-7394027077984723-ch1.bin')},
 {'label': 0,
  'channels': 1,
  'interval': [0, 7031250],
  'bin_path': PosixPath('/mnt/home2/Motor_project/AE_PETR_loziska/01/AE-DATA-conti-7394027035168519-ch1.bin')},
 {'label': 4,
  'channels': 1,
  'interval': [0, 7031250],
  'bin_path': PosixPath('/mnt/home2/Motor_project/AE_PETR_loziska/20/AE-DATA-conti-739

In [4]:
train_set = SignalDataset(step=1000, window_size=1000, bin_setup=train_config, source_dtype="float32")
test_set = SignalDataset(step=1000, window_size=1000, bin_setup=test_config, source_dtype="float32")

In [6]:
len(train_set)

295302

In [ ]:
nn_config["training_params"]["lr"] = config["lr"]
nn_config["training_params"]["dataloader_params"]["batch_size"] = config["batch_size"]
update_layer_argument(nn_config, 'linear1', 'out_features', config['linear1'])
update_layer_argument(nn_config, 'linear2', 'in_features', config['linear1'])
update_layer_argument(nn_config, 'linear2', 'out_features', config['linear2'])
update_layer_argument(nn_config, 'linear3', 'in_features', config['linear2'])


config = {
    "lstm": {
        "hidden_size": tune.randint(128,1500),
        "num_layers": tune.choice([1, 2, 3])},
    "linear1": tune.randint(512,2000),
    "linear2": tune.randint(200, 1500),
    "lr": tune.uniform(1e-4, 1e-1),
    "warmups": tune.choice([1, 2, 3]),
    "dropout": tune.uniform(0, 0.5),
    "batch_size": tune.choice([128, 256, 512, 1024]),
    "epoch_num": tune.choice([5, 10, 15, 20, 40]),
    "window_size" : tune.randint(500, 1000000),
}

In [ ]:
config = {
    "linear1": tune.choice([2 ** i for i in range(10, 14)]),
    "linear2": tune.choice([2 ** i for i in range(8, 12)]),
    "lr": tune.loguniform(1e-4, 1e-2),
    "batch_size": tune.choice([128, 256, 512])
}

In [5]:
nn_config = load_yaml(Path("nn_yaml_configs/CNN_spec.yaml"))

In [6]:
def update_layer_argument(nn_config: dict, layer_id: str, arg: str, value):
    layers_configs = nn_config["model"]["kwargs"]["layers"]
    if isinstance(layers_configs, list):
        for layer in layers_configs:
            if layer.get("id") == layer_id:
                layer["kwargs"][arg] = value

    elif isinstance(layers_configs, dict):
        for name, layer_config in layers_configs.items():
            for layer in layer_config:
                if layer.get("id") == layer_id:
                    layer["kwargs"][arg] = value

In [ ]:
    update_layer_argument(nn_config, 'conv1', 'out_channels', config['first_conv'])
    # update_layer_argument(nn_config, 'conv1', 'kernel_size', config['kernel_size'])
    # update_layer_argument(nn_config, 'conv1', 'padding', config['kernel_size'] // 2)

    update_layer_argument(nn_config, 'bn1', 'num_features', config['first_conv'])

    update_layer_argument(nn_config, 'conv2', 'in_channels', config['first_conv'])
    update_layer_argument(nn_config, 'conv2', 'out_channels', config['second_conv'])
    # update_layer_argument(nn_config, 'conv2', 'kernel_size', config['kernel_size'])
    # update_layer_argument(nn_config, 'conv2', 'padding', config['kernel_size'] // 2)

    update_layer_argument(nn_config, 'bn2', 'num_features', config['second_conv'])

    update_layer_argument(nn_config, 'conv3', 'in_channels', config['second_conv'])
    # update_layer_argument(nn_config, 'conv3', 'kernel_size', config['kernel_size'])
    # update_layer_argument(nn_config, 'conv3', 'padding', config['kernel_size'] // 2)
    update_layer_argument(nn_config, 'conv3', 'out_channels', config['third_conv'])

In [14]:
out_channel_values_first = [32, 64, 128]
out_channel_values_second = [64, 128, 256]
kernel_sizes = [3, 5]

# Iterating through the parameters
for out_channels1 in out_channel_values_first:
    for out_channels2 in out_channel_values_second:
        for kernel_size1 in kernel_sizes:
            for kernel_size2 in kernel_sizes:
                for kernel_size3 in kernel_sizes:
                    # Update the configuration for the current combination of parameters
                    update_layer_argument(nn_config, 'conv1', 'out_channels', out_channels1)
                    update_layer_argument(nn_config, 'conv1', 'kernel_size', kernel_size1)
                    update_layer_argument(nn_config, 'conv1', 'padding', kernel_size1 // 2)
                    
                    update_layer_argument(nn_config, 'bn1', 'num_features', out_channels1)
                    
                    update_layer_argument(nn_config, 'conv2', 'out_channels', out_channels2)
                    update_layer_argument(nn_config, 'conv2', 'kernel_size', kernel_size2)
                    update_layer_argument(nn_config, 'conv2', 'padding', kernel_size2 // 2)
                    update_layer_argument(nn_config, 'conv2', 'in_channels', out_channels1)
                    
                    update_layer_argument(nn_config, 'bn2', 'num_features', out_channels2)
                    
                    update_layer_argument(nn_config, 'conv3', 'kernel_size', kernel_size3)
                    update_layer_argument(nn_config, 'conv3', 'padding', kernel_size3 // 2)
                    update_layer_argument(nn_config, 'conv3', 'in_channels', out_channels2)
                    neuro_net = NeuroNet(config=nn_config, tensorboard=True)
                    neuro_net.train(train_set)

In [15]:
def train_model(config, dataset):
    update_layer_argument(nn_config, 'conv1', 'out_channels', config['first_conv'])
    # update_layer_argument(nn_config, 'conv1', 'kernel_size', config['kernel_size'])
    # update_layer_argument(nn_config, 'conv1', 'padding', config['kernel_size'] // 2)
    
    update_layer_argument(nn_config, 'bn1', 'num_features', config['first_conv'])
    
    update_layer_argument(nn_config, 'conv2', 'in_channels', config['first_conv'])
    update_layer_argument(nn_config, 'conv2', 'out_channels', config['second_conv'])
    # update_layer_argument(nn_config, 'conv2', 'kernel_size', config['kernel_size'])
    # update_layer_argument(nn_config, 'conv2', 'padding', config['kernel_size'] // 2)

    update_layer_argument(nn_config, 'bn2', 'num_features', config['second_conv'])
    
    update_layer_argument(nn_config, 'conv3', 'in_channels', config['second_conv'])   
    # update_layer_argument(nn_config, 'conv3', 'kernel_size', config['kernel_size'])
    # update_layer_argument(nn_config, 'conv3', 'padding', config['kernel_size'] // 2)
    update_layer_argument(nn_config, 'conv3', 'out_channels', config['third_conv'])

    neuro_net = NeuroNet(config=nn_config, tensorboard=True)
    neuro_net.train(dataset)

In [ ]:
class model_optimizer():
    def __init__(self, model: SignalModel):
        pass
    def load_params(self, params_grid: dict):
        self.params = params_grid